# MeshCat.jl Tutorial
### Create animations in Julia

In [23]:
using Blink # Custom size window
using Colors: RGBA, RGB # Handle RGB colors
using CoordinateTransformations # Translations and rotations
using FileIO # Save and load files
using GeometryTypes: # Define geometric shapes
    GeometryTypes, HyperRectangle, Vec, Point, Rectangle, Cylinder,
    HomogenousMesh, SignedDistanceField, HyperSphere, GLUVMesh, Pyramid
using JLD2 # Save and load Julia data structures
using LinearAlgebra
using MeshCat # Visualize 3D animations
using MeshIO # Load meshes in MeshCat

In [24]:
include("../src/structure.jl") # Import parameters data structures
include("../src/parameters.jl"); # Import default parameters of the scene

### Load the trajectories of the cars and the parameters of the scene obtained via ALGAMES.

In [25]:
function load_data()
    load("../animation/trajectory/scene.jld2", "X", "U",
        "problem_parameters", "scene_parameters")
end
X, U, prob_param, scene_param = load_data()

# Extract problem parameters
N = prob_param.N
p = prob_param.p
radius = prob_param.radius
state_bound_points = prob_param.boundary_points
merge_point = prob_param.merge_point

# Extract scene parameters
road_dims = scene_param.road_dims
line_width = scene_param.line_width
boundary_width = scene_param.boundary_width
road_trans = scene_param.road_trans
road_translation = Translation(road_trans...)
boundary_height = scene_param.boundary_height
cylinder_height = scene_param.cylinder_height
car_offset = scene_param.car_offset
color_names = scene_param.color_names
color_values = scene_param.color_values;
tracking_id = scene_param.tracking_id
zoom = scene_param.zoom
traveling_trans = scene_param.traveling_trans
traveling_rot = scene_param.traveling_rot
birdseye_trans = scene_param.birdseye_trans
birdseye_rot = scene_param.birdseye_rot;

### For each car, create a sequence of car translations and rotations corresponding to their trajectories. 

In [27]:
function car_transformations(X, prob_param, scene_param)
    car_offset = scene_param.car_offset
    N = prob_param.N
    p = prob_param.p
    car_translations = []
    car_rotations = []
    for k=1:N
        car_translation = [Translation(X[1:2,k,id]..., 0) for id=1:p]
        if k==N
            # Deals with the last step, we assume constant heading
            car_rotation = car_rotations[end]
        else
            angles = [atan(X[2,k+1,id]-X[2,k,id], X[1,k+1,id]-X[1,k,id])
                for id=1:p]
            car_rotation = [LinearMap(AngleAxis(angles[id], 0, 1, 0))
                for id=1:p]
        end
        push!(car_translations, car_translation)
        push!(car_rotations, car_rotation)
    end
    return car_translations, car_rotations
end
car_translations, car_rotations = car_transformations(
        X, prob_param, scene_param);

# Create the scene

### Create and open visualizer

In [28]:
vis = Visualizer()
open(vis);

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8703
└ @ MeshCat /home/simon/.julia/packages/MeshCat/GMobF/src/servers.jl:24
┌ Error: error handling request
│   exception = (Base.IOError("stream is closed or unusable", 0), Base.StackTraces.StackFrame[check_open at stream.jl:323 [inlined], uv_write_async(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64, ::Task) at stream.jl:901, uv_write(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:877, unsafe_write(::Sockets.TCPSocket, ::Ptr{UInt8}, ::UInt64) at stream.jl:931, macro expansion at ConnectionPool.jl:134 [inlined], write at io.jl:177 [inlined], closebody at Streams.jl:111 [inlined], closewrite(::HTTP.Streams.Stream{HTTP.Messages.Request,HTTP.ConnectionPool.Transaction{Sockets.TCPSocket}}) at Streams.jl:126, (::getfield(HTTP.Servers, Symbol("##13#14")){getfield(WebSockets, Symbol("#_servercoroutine#11")){WebSockets.ServerWS},HTTP.ConnectionPool.Transaction{Sockets.TCPSocket},HTTP.Streams.Stream{HTTP.Messages.Request,HTTP.Conn

### Display cars in Meshcat
- Load the Mesh from the .obj file.
- Load the texture image from the .png file.
- Create texture from the image.
- Create material from the texture.
- Create object by combining the material and the mesh. 


In [29]:
# Load mesh
car = load("../animation/object/car/car_geometry.obj", GLUVMesh)
for id=1:p
    # Load texture image
    car_image = PngImage("../animation/image/" * color_names[id] * ".png")
    # Create texture 
    car_texture = Texture(image=car_image)
    # Create material
    car_material = MeshLambertMaterial(map=car_texture)
    # Create object
    setobject!(vis["car$id"], car, car_material)
    # Place object at the origin
    settransform!(vis["car$id"], car_offset)
    # Add collision avoidance cylinders
    collision_cylinder = Cylinder(Point(0.0,0.0,0.0),
        Point(0.0,0.0,cylinder_height), radius)
    cylinder_material = MeshPhongMaterial(color=RGBA(
            color_values[id,:]..., 9e-1))
    setobject!(vis["collision_cylinder$id"], 
        collision_cylinder, cylinder_material)
    settransform!(vis["collision_cylinder$id"], 
        Translation(0.0, 0.0, 0.0))
end

### Display the main roadway

In [30]:
road_image = PngImage("../animation/image/road.png")
road_texture = Texture(image=road_image)
road_material = MeshLambertMaterial(map=road_texture)
road_width = (state_bound_points[1,2]+radius) - 
    (state_bound_points[3,2]-radius) + 2*line_width
road = HyperRectangle(Vec(0.0, -(road_width/2-road_dims[2]), 
    -road_dims[3]), Vec(road_dims[1], road_width, road_dims[3]))
setobject!(vis["road"], road, road_material)
settransform!(vis["road"], road_translation);

### Display the yellow lines

In [31]:
line_material = MeshPhongMaterial(color=RGBA(1, 1, 0, 1.0))
line = Rectangle(0.0, 0.0, road_dims[1], line_width)
for i=1:3
    setobject!(vis["line$i"], line, line_material)
    line_translation = compose(Translation(0, (i-1)*road_dims[2] - 
            line_width/2, 0.04), road_translation)
    settransform!(vis["line$i"], line_translation)
end

### Display the road boundaries

In [32]:
boundary_image = PngImage("../animation/image/black_boundary.png")
boundary_texture = Texture(image=boundary_image)
boundary_material = MeshLambertMaterial(map=boundary_texture)
ramp_extension = 10.0;

#### Upper boundary


In [33]:
boundary1 = HyperRectangle(Vec(0.0, 0.0, 0.0), 
    Vec(road_dims[1], boundary_width, boundary_height))
setobject!(vis["boundary1"], boundary1, boundary_material)
boundary1_translation = compose(Translation(0, 
        state_bound_points[1,2]+radius, 0.0), road_translation)
settransform!(vis["boundary1"], boundary1_translation);

#### Lower boundary 1


In [34]:
Δ = [merge_point, state_bound_points[3,2] - state_bound_points[2,2]]
length = sqrt(Δ'*Δ) + ramp_extension
angle = atan(Δ[2], Δ[1])
boundary2 = HyperRectangle(Vec(-ramp_extension, 
        -boundary_width, 0.0), Vec(length - radius*sin(angle/2), 
        boundary_width, boundary_height))
setobject!(vis["boundary2"], boundary2, boundary_material)
boundary2_translation = Translation(radius*cos(-pi/2+angle), 
    state_bound_points[2,2]+radius*sin(-pi/2+angle), 0.0)
boundary2_translation = compose(Translation(-4*radius*sin(angle/2), 0, 0),
    boundary2_translation)
boundary2_translation = compose(LinearMap(AngleAxis(angle, 0, 0, 1)), 
    boundary2_translation)
settransform!(vis["boundary2"], boundary2_translation);

#### Lower boundary 2


In [35]:
boundary3 =	HyperRectangle(Vec(0.0, -boundary_width, 0.0), 
    Vec(road_dims[1]-merge_point+road_trans[1]-radius*sin(angle/2), 
        boundary_width, boundary_height))
setobject!(vis["boundary3"], 
    boundary3, boundary_material)
boundary3_translation = Translation(merge_point+radius*sin(angle/2), 
    state_bound_points[3,2]-radius, 0.0)
settransform!(vis["boundary3"], boundary3_translation);

### Display the ramp

In [36]:
ramp_width = road_width - road_dims[2]
ramp = HyperRectangle(Vec(-ramp_extension, 0.0, -road_dims[3]), 
    Vec(length, ramp_width, road_dims[3]))
ramp_translation = Translation(0.0, state_bound_points[2,2]-radius, -0.04)
ramp_translation = compose(LinearMap(AngleAxis(angle, 0, 0, 1)), 
    ramp_translation)
setobject!(vis["ramp"], ramp, road_material)
settransform!(vis["ramp"], ramp_translation);

# Improve the visualization
### Delete the grid and the axes in the visualizer.

In [37]:
delete!(vis["/Grid"])
delete!(vis["/Axes"]);

### Set the camera and light properties.

In [38]:
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 0.8)
setprop!(vis["/Lights/DirectionalLight/<object>"], "intensity", 1.2);

# Animate the scene
- Create an animation 
- Apply cars' rotations and translations at each frame
- Apply rotations and translations to the camera

### Create an animation and set the frame rate.

In [39]:
framerate = 6
anim = MeshCat.Animation(
    Dict{MeshCat.SceneTrees.Path,MeshCat.AnimationClip}(), 
    framerate);

In [43]:
animation_type = :image
tracking_id = 1;

### Set the rotations and translations of the camera and the cars for each frame. 

In [46]:
for k=1:N-1
    # Set the poses of the two cars.
    MeshCat.atframe(anim, vis, k) do frame
        for id=1:p
            car_translations[k][id]
            car_rotations[k][id]
            settransform!(frame["car$id"],
                compose(compose(car_translations[k][id],
                car_offset),car_rotations[k][id]))
            settransform!(frame["collision_cylinder$id"], 
                car_translations[k][id])
        end
        setprop!(frame["/Cameras/default/rotated/<object>"], "zoom", zoom)
        setprop!(frame["/Cameras/default/rotated/<object>"], "zoom", zoom)
        if animation_type == :video
             # follow car with id tracking_id
            camera_transformation = compose(compose(
                Translation(X[1:2,k,tracking_id]..., 0),
                traveling_trans),
                traveling_rot
                )
        elseif animation_type == :image
            camera_transformation = compose(
                birdseye_trans,
                birdseye_rot)
        end
        settransform!(frame["/Cameras/default"], camera_transformation)
    end
end;

┌ Warning: atframe(f::Function, anim::Animation, vis::Visualizer, frame::Integer) is deprecated.
│ Please use atframe(g, anim, frame) instead, where g is similar to f but takes
│ no arguments and should call methods on vis.
│ See also the updated animation example notebook.
│   caller = top-level scope at In[46]:3
└ @ Core In[46]:3


### Animate

In [47]:
MeshCat.setanimation!(vis, anim);

# Create a video 
- Go to the browser and record the animation: "Open Controls/Recording/record"
- Using convert_meshcat_to_video to get the video. 

In [21]:
function convert_meshcat_to_video(;filename=nothing,
    input_path="results/meshcat_sequence/",
    output_path="results/video/")
    # Saving MeshCat sequence as a video.
    meshcat_sequence_dir = joinpath(@__DIR__, "..", input_path)
    if filename==nothing
        filenames = readdir(meshcat_sequence_dir)
    else
        filenames = [filename * ".tar"]
    end
    for filename in filenames
        println("Converting " * filename * " to video." )
        video_dir = joinpath(@__DIR__, "..", output_path, 
            filename[1:end-4] * ".mp4",)
        MeshCat.convert_frames_to_video(
            meshcat_sequence_dir * filename,
            video_dir,
            overwrite=true)
    end
    return
end

convert_meshcat_to_video (generic function with 1 method)

In [48]:
convert_meshcat_to_video()

Converting meshcat_1575256500546.tar to video.


ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena